In [3]:
import os
import numpy as np
import pandas as pd
import nltk
from sklearn import metrics
import sklearn 
import string
import re # helps you filter urls
from sklearn.metrics import accuracy_score
import os
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd
import math
import numpy as np
from math import sin, cos, sqrt, atan2, radians
import pandas as pd
import datetime
import datetime
from datetime import datetime, time, timedelta
import pandas as pd
import numpy as np
import networkx as nx
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from os import listdir
from os.path import isfile, join
from calendar import monthrange
import time
import timeit
import sklearn
from sklearn.neighbors import DistanceMetric
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import RandomOverSampler
from pandas import read_csv
from imblearn.over_sampling import SMOTE
from collections import Counter
from matplotlib import pyplot
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_breast_cancer 
from sklearn.svm import SVC 
import sklearn.metrics as metrics
import nltk

In [4]:
def process(test, lemmatizer=nltk.stem.wordnet.WordNetLemmatizer()):
    clean = re.compile('<.*?>')
    test=re.sub(clean, '', test)
    posMapping = {# "First_Letter by nltk.pos_tag":"POS_for_lemmatizer"
            "N":'n',
            "V":'v',
            "J":'a',
            "R":'r'}
    url_reg='http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    lemmatizer=nltk.stem.wordnet.WordNetLemmatizer()
    words=re.sub(url_reg, '', test)
    words=words.replace('\'s', '')
    words=words.replace('\'', '')
    words = [word.lower() for word in words]  
    listnew=[]
    for i in words:
          if i in string.punctuation:
              listnew.append(" ")
          else:
              listnew.append(i)        
    words="".join(listnew)
    words=nltk.word_tokenize(words)
    def lemmatizerfunction(word,pos):
          if pos==None:
              return(None)
          else:
              result=lemmatizer.lemmatize(word,pos)
              return(result)  
    
    list1=nltk.pos_tag(words)
    
    list1=[ele[1] for ele in list1]   
    list1=[ele[0] for ele in list1]     
    def converter(x):
        if x in posMapping:
            return(posMapping[x])
        else:     
            return('n')      
    list1=[converter(i) for i in list1]  
    listfinal=list(zip(words,list1))
    listfinal=[lemmatizerfunction(i[0],i[1]) for i in listfinal]
    return(listfinal) 

In [5]:
def process_all(df, lemmatizer=nltk.stem.wordnet.WordNetLemmatizer()):
    df['text']=df.apply(lambda row : process(row['text'],lemmatizer), axis = 1) 
    return(df)

In [6]:
def create_features(processed_tweets, stopwords):
    from sklearn.feature_extraction.text import TfidfVectorizer
    vectorizer = TfidfVectorizer(stop_words=stopwords,min_df=2,ngram_range=(1,2))
    corpus=[" ".join(review) for review in processed_tweets['text'].values]
    sm=vectorizer.fit_transform(corpus)
    return(vectorizer,sm)

def create_labels(processed_tweets):
    df=processed_tweets
    Y=df['sentiment']
    return(Y)


def learn_classifier(X_train, y_train, kernel,C,gamma):
    clf = SVC(kernel=kernel, C=C,gamma=gamma)
    #print(kernel)
    y_train=y_train.astype('int')
    clf.fit(X_train,y_train)
    return(clf)

# execute code
def evaluate_classifier(classifier, X_validation, y_validation):
    y_validation=pd.to_numeric(y_validation)
    predictions=classifier.predict(X_validation)
    
    actuals=y_validation
    Z=list(zip(actuals,predictions))
    
    from sklearn.metrics import accuracy_score
    print(predictions,actuals)
    
    # Print the precision and recall, among other metrics
    print(metrics.classification_report(predictions,actuals))
    
    return(accuracy_score(predictions,actuals))

In [42]:
path=r'D:\SentimemtAnalysis\SentimentAnalysis'
os.chdir(path)
onlyfiles = [f for f in listdir(path) if isfile(join(path, f))]
filename='train.csv'
lemmatizer=nltk.stem.wordnet.WordNetLemmatizer()
stopwords=nltk.corpus.stopwords.words('english')

In [43]:
df=pd.read_csv(filename)
df.head()

,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative


In [44]:
df.columns=['ID','text','selected_text','sentiment']
df=df[['text','sentiment']]
df=df.replace({'positive': 1,'negative':-1,'neutral':0})
df=df.loc[df['sentiment'].isin([1,-1,0])]
df.head()

,text,sentiment
0,"I`d have responded, if I were going",0
1,Sooo SAD I will miss you here in San Diego!!!,-1
2,my boss is bullying me...,-1
3,what interview! leave me alone,-1
4,"Sons of ****, why couldn`t they put them on t...",-1


In [45]:
df=df.reset_index() 
df=df.dropna(subset=['text', 'sentiment'])
processed_tweets = process_all(df)
processed_tweets.head()

,index,text,sentiment
0,0,"[i, d, have, respond, if, i, be, go]",0
1,1,"[sooo, sad, i, will, miss, you, here, in, san,...",-1
2,2,"[my, bos, be, bully, me]",-1
3,3,"[what, interview, leave, me, alone]",-1
4,4,"[son, of, why, couldn, t, they, put, them, on,...",-1


In [46]:
(tfidf, X) = create_features(processed_tweets, stopwords)

y = create_labels(processed_tweets)

In [47]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)
y_train=y_train.astype('int')

In [48]:
from sklearn.model_selection import GridSearchCV
param_grid=[{
        'C':[0.01,0.1,1,10,100,1000,10000],
        'penalty':["l1","l2"], 'solver':['liblinear'],
        'multi_class':['ovr','auto'],
        'max_iter':[1000] , 'class_weight': ['balanced',None]},
    {
        'C':[0.01,0.1,1,10,100,1000,10000],
        'penalty':["l2"], 'solver':['newton-cg','sag','saga','lbfgs'],
        'multi_class':['ovr','auto','multinomial'],
        'max_iter':[500], 'class_weight': ['balanced',None]
    }]

print("strting grid search")  
logreg_cv=GridSearchCV( LogisticRegression(), param_grid, verbose = 3, cv=3, n_jobs = 16)
logreg_cv.fit(X_train,y_train)
print("tuned hpyerparameters :(best parameters) ",logreg_cv.best_params_)
print("accuracy :",logreg_cv.best_score_)

strting grid search
Fitting 3 folds for each of 224 candidates, totalling 672 fits
tuned hpyerparameters :(best parameters)  {'C': 1, 'class_weight': 'balanced', 'max_iter': 1000, 'multi_class': 'auto', 'penalty': 'l1', 'solver': 'liblinear'}
accuracy : 0.6978436804084515


In [49]:
best_params = logreg_cv.best_params_
print(best_params)
logreg2=LogisticRegression(C=best_params['C'],penalty=best_params['penalty'],max_iter= best_params['max_iter'], multi_class=best_params['multi_class'], solver=best_params['solver'], class_weight=best_params['class_weight'])
logreg2.fit(X_train,y_train)
pred = logreg2.predict(X_test)
print(metrics.classification_report(pred,y_test))

{'C': 1, 'class_weight': 'balanced', 'max_iter': 1000, 'multi_class': 'auto', 'penalty': 'l1', 'solver': 'liblinear'}
              precision    recall  f1-score   support

          -1       0.61      0.72      0.66      2196
           0       0.74      0.66      0.69      4107
           1       0.74      0.76      0.75      2766

    accuracy                           0.70      9069
   macro avg       0.70      0.71      0.70      9069
weighted avg       0.71      0.70      0.70      9069



In [50]:
def best_model_selection_report():
    mainlist=[]

    for parameters in param_grid:
        for optimizer in parameters['solver']:
            for multiclass in parameters['multi_class']:
                for penalty in parameters['penalty']:
                    for c in parameters['C']:
                        logreg_clfr = LogisticRegression(C=c ,penalty=penalty,max_iter= 1000, multi_class=multiclass, solver=optimizer,  class_weight='balanced')
                        logreg_clfr.fit(X_train,y_train)
                        accuracy = evaluate_classifier(logreg_clfr, X_test, y_test)
                        mainlist.append((optimizer,multiclass,penalty,c,accuracy))
    return mainlist

report = best_model_selection_report()
print(report)

   3909
           1       0.72      0.75      0.74      2735

    accuracy                           0.69      9069
   macro avg       0.68      0.69      0.69      9069
weighted avg       0.69      0.69      0.69      9069

[0 1 0 ... 0 0 1] 2707     0
24003    1
8172     0
3081     0
1693     1
        ..
26578    0
23652    0
21585    0
22847    0
16394    1
Name: sentiment, Length: 9069, dtype: int64
              precision    recall  f1-score   support

          -1       0.63      0.65      0.64      2509
           0       0.65      0.63      0.64      3773
           1       0.72      0.73      0.73      2787

    accuracy                           0.67      9069
   macro avg       0.67      0.67      0.67      9069
weighted avg       0.67      0.67      0.67      9069

[0 1 1 ... 0 0 1] 2707     0
24003    1
8172     0
3081     0
1693     1
        ..
26578    0
23652    0
21585    0
22847    0
16394    1
Name: sentiment, Length: 9069, dtype: int64
              precision    

In [53]:
results=pd.DataFrame(report)
results.columns=['optimizer','multiclass','penalty','c','accuracy']
results= results.groupby(['optimizer','multiclass','penalty','c']).agg({'accuracy': ['mean', 'min', 'max']})
results.columns = ['accuracy_mean', 'accuracy_min', 'accuracy_max']
results=pd.DataFrame(results)
results=results.reset_index()
results.to_excel(r'D:\SentimemtAnalysis\SentimentAnalysis\Ajay_codes\LogReg_report.xlsx', index = False)
print(results)

    optimizer multiclass penalty         c  accuracy_mean  accuracy_min  \
0       lbfgs       auto      l2      0.01       0.654758      0.654758   
1       lbfgs       auto      l2      0.10       0.677804      0.677804   
2       lbfgs       auto      l2      1.00       0.688830      0.688830   
3       lbfgs       auto      l2     10.00       0.658066      0.658066   
4       lbfgs       auto      l2    100.00       0.609439      0.609439   
..        ...        ...     ...       ...            ...           ...   
107      saga        ovr      l2      1.00       0.685412      0.685412   
108      saga        ovr      l2     10.00       0.666777      0.666777   
109      saga        ovr      l2    100.00       0.619363      0.619363   
110      saga        ovr      l2   1000.00       0.585291      0.585291   
111      saga        ovr      l2  10000.00       0.575918      0.575918   

     accuracy_max  
0        0.654758  
1        0.677804  
2        0.688830  
3        0.658066  